<a href="https://colab.research.google.com/github/wolffg777/descriptors_to_music_rec/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM

- see what happens with more layers  
- word2vec on descriptors and genres
  - first just working
  - then working out OOV (out of vocabulary) problems
- testing it to see improvement
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)



In [ ]:
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy

Saving kaggle.json to kaggle.json
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
  0% 0.00/863k [00:00<?, ?B/s]
100% 863k/863k [00:00<00:00, 111MB/s]
Archive:  rym-top-5000.zip
  inflating: rym_clean1.csv          
  inflating: rym_raw1.csv            
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('rym_clean1.csv')
# df.head(1)

# Remove rating_count, review_count, and avg_rating columns
df = df.drop(columns=['avg_rating'])
df = df.drop(columns=['rating_count'])
df = df.drop(columns=['review_count'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['position'])
df = df.drop(columns=['release_type'])
df.head(5)

,release_name,artist_name,release_date,primary_genres,secondary_genres,descriptors
0,OK Computer,Radiohead,1997-06-16,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,Radiohead,2000-10-03,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,Pink Floyd,1973-03-23,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi..."
3,Loveless,My Bloody Valentine,1991-11-11,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist..."


In [ ]:
# Preprocessing Data

# One-hot-encoding

df['descriptors'] = df['descriptors'].str.split(',')
desc_encoded = df['descriptors'].str.join('|').str.get_dummies()
desc_frequencies = desc_encoded.iloc[:, 2:].sum()

df['primary_genres'] = df['primary_genres'].str.split(',')
pg_encoded = df['primary_genres'].str.join('|').str.get_dummies()
pg_frequencies = pg_encoded.iloc[:, 2:].sum()

df['secondary_genres'] = df['secondary_genres'].str.split(',')
sg_encoded = df['secondary_genres'].str.join('|').str.get_dummies()
sg_frequencies = sg_encoded.iloc[:, 2:].sum()

In [ ]:
rarity_threshold_desc = 5
desc_encoded.columns = desc_encoded.columns.str.strip()
rare_descriptors = desc_frequencies[desc_frequencies < rarity_threshold_desc]

print(len(rare_descriptors))

60


In [ ]:
rarity_thresholdpg = 4
pg_encoded.columns = pg_encoded.columns.str.strip()
rare_primary_genres = pg_frequencies[pg_frequencies < rarity_thresholdpg]

print(len(rare_primary_genres))

381


In [ ]:
rarity_thresholdsg = 2
sg_encoded.columns = sg_encoded.columns.str.strip()
rare_secondary_genres = sg_frequencies[sg_frequencies < rarity_thresholdsg]

print(len(rare_secondary_genres))

316


In [ ]:
desc_to_remove = desc_frequencies[desc_frequencies < rarity_threshold_desc].index.str.strip()
dfdesc_filtered = desc_encoded.drop(columns = desc_to_remove)

pgs_to_remove = pg_frequencies[pg_frequencies < rarity_thresholdpg].index.str.strip()
dfpg_filtered = pg_encoded.drop(columns = pgs_to_remove)

sgs_to_remove = sg_frequencies[sg_frequencies < rarity_thresholdsg].index.str.strip()
dfsg_filtered = sg_encoded.drop(columns = sgs_to_remove)

# dfpg_encoded = pd.concat([df, dfpg_filtered], axis=1)
# dfpg_encoded = dfpg_encoded.drop(columns=['primary_genres'])

# dfsg_encoded = pd.concat([dfpg_encoded, dfsg_filtered], axis=1)
# df_pg_sg_encoded = dfsg_encoded.drop(columns=['secondary_genres'])

df_genres1hot = pd.concat([dfdesc_filtered, dfpg_filtered], axis=1)
df_genres1hot = pd.concat([df_genres1hot, dfsg_filtered], axis=1)

# df_final.info().head(3)

In [ ]:
df_genres1hot.head(3)

,Christian,LGBT,Wallof Sound,abstract,acoustic,aggressive,alcohol,alienation,anarchism,androgynousvocals,...,Turkish Folk Music,Turntablism,Twee Pop,UKBass,UKHip Hop,USPower Metal,Vocal Jazz,Witch House,Wonky,Zolo
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# PREPROCESSING ALBUM NAMES,GENRES, AND RELEASE DATES


In [ ]:
release_names = df["release_name"]
artist_names = df["artist_name"]
release_dates = df["release_date"]

In [ ]:
release_names_encoded = pd.get_dummies(release_names, columns=["release_name"])
artist_names_encoded = pd.get_dummies(artist_names, columns=["artist_name"])

df_album_info_encoded = pd.concat([release_names_encoded, artist_names_encoded], axis=1)
df_album_info_encoded.head(5)

,"""Awaken, My Love!""","""Born Into Trouble as the Sparks Fly Upward.""","""Charlie""","""Disco!""","""Heroes""","""Love and Theft""","""Luciferian Towers""","""The Spaghetti Incident?""","""Them""","""This Is Our Punk-Rock,"" Thee Rusted Satellites Gather + Sing,",...,福居良 [Ryo Fukui],細野晴臣 [Haruomi Hosono] / 鈴木茂 [Shigeru Suzuki] / 山下達郎 [Tatsuro Yamashita],青葉市子 [Ichiko Aoba],高木正勝 [Masakatsu Takagi],공중도덕,공중도둑 [Mid-Air Thief],유키카 [YUKIKA],이달의 소녀 오드아이써클 [LOONA ODD EYE CIRCLE],파란노을 [Parannoul],파란노을 [Parannoul] / Asian Glow / sonhos tomam conta
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
 # TIME TO START PREDICTING

# Perform train/test split using scikit-learn
X_train, X_test, y_train, y_test = train_test_split(df_genres1hot, df_album_info_encoded, test_size=0.2, random_state=42)

# Create TensorFlow datasets from the NumPy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Define batch size for training and testing
batch_size = 32

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)


In [ ]:
# Define your model
model = keras.Sequential()

# Input layer
model.add(layers.Input(shape=(1286,)))  # Assuming 1286 input features

# Hidden layers
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))

# Output layer
model.add(layers.Dense(7028, activation='softmax'))  # 7028 output neurons for descriptors and genres

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               329472    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 7028)              906612    
                                                                 
Total params: 1268980 (4.84 MB)
Trainable params: 1268980 (4.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 32

# Assuming you have defined your model as shown in your previous code

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Use model.fit with the training dataset
model.fit(train_dataset, epochs=epochs)


Epoch 1/10
125/125 [==============================] - 3s 15ms/step - loss: 17.7265 - accuracy: 0.0000e+00
Epoch 2/10
125/125 [==============================] - 2s 15ms/step - loss: 17.6497 - accuracy: 0.0000e+00
Epoch 3/10
125/125 [==============================] - 2s 18ms/step - loss: 17.5833 - accuracy: 0.0000e+00
Epoch 4/10
125/125 [==============================] - 2s 14ms/step - loss: 17.1748 - accuracy: 0.0000e+00
Epoch 5/10
125/125 [==============================] - 2s 14ms/step - loss: 16.0415 - accuracy: 2.5000e-04
Epoch 6/10
125/125 [==============================] - 2s 15ms/step - loss: 14.2636 - accuracy: 5.0000e-04
Epoch 7/10
125/125 [==============================] - 2s 14ms/step - loss: 12.7408 - accuracy: 0.0015
Epoch 8/10
125/125 [==============================] - 2s 17ms/step - loss: 11.4970 - accuracy: 0.0037
Epoch 9/10
125/125 [==============================] - 2s 17ms/step - loss: 10.4484 - accuracy: 0.0082
Epoch 10/10
125/125 [==============================] - 2s 

In [ ]:
# Assuming X_test is your test data
predictions = model.predict(train_dataset)

125/125 [==============================] - 1s 5ms/step


In [ ]:
# Assuming test_dataset is your testing dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

32/32 [==============================] - 0s 9ms/step - loss: 22.3482 - accuracy: 0.0000e+00
Test Loss: 22.348201751708984, Test Accuracy: 0.0


things to do: stricter filtering of descriptors
one-hot the genres, word2vec (number of dimensions depends on embedding model, basic word2vec is 30 - 100), the descriptors and average the value and put it in space. tutorials on youtube seem good.

try to find a taxonomy of genres online, if not split up words and vaguely separating the musical words

Partial credit accuracy/reward system where "radiohead" for atmospheric even if the album recommended is not atmospheric

In [ ]:
import os
import base64
from requests import post, get
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


os.environ['SPOTIPY_CLIENT_ID'] = '08a0640020ff4e5d9713aaeacfbd1886'
os.environ['SPOTIPY_CLIENT_SECRET'] = '81cefe24f94b4f24af9fe1ad2e70ef4a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://example.com/callback'

#Authorization Code Flow
scopes = "user-library-read user-top-read user-follow-read"

sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          scope=scopes, open_browser=False))

In [ ]:
#Get Specified User's Playlists
playlists = sp.user_playlists('bpostley07')

while playlists:
    for i, playlist in enumerate(playlists['items']):
        #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))

    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


Go to the following URL: https://accounts.spotify.com/authorize?client_id=08a0640020ff4e5d9713aaeacfbd1886&response_type=code&redirect_uri=http%3A%2F%2Fexample.com%2Fcallback&scope=user-library-read+user-top-read+user-follow-read
Enter the URL you were redirected to: http://example.com/callback?code=AQDT_1hWVCfp8OTMzWZMg9DffCr8lSbbsWP3QIZNpErpIR28dIzPS1v3uYIt6MPxONNuUynIGIKigQDCiCUP2Y5t-ooHX3e89o5w9pARPsGQFme79z7Bo_N6FlTzICYNZz36cwRe1UDczZVrZ6EcXIsJ4NYUis3X2VmfrypaDuLL-IrzOmPZ7kkRXVchfYEw2owltdqo3uGiFDSyNf3aPEt_9ONEB9XeAe5E2QMUoBt7nyc
   1 Trending Music
   2 Griffin-sept-18
   3 7/11/18
   4 2/25/18
   5 9/16/17
   6 you know
   7 JCS
   8 8/21/16


In [ ]:
album_name = "OK Computer"
artist_name = "Radiohead"

# Search for the album
results = sp.search(q=f"album:{album_name} artist:{artist_name}", type="album")

album_id = results['albums']['items'][0]['id']

album_tracks = sp.album_tracks(album_id)

for track in album_tracks['items']:
    print(track['name'])


Airbag
Paranoid Android
Subterranean Homesick Alien
Exit Music (For A Film)
Let Down
Karma Police
Fitter Happier
Electioneering
Climbing Up the Walls
No Surprises
Lucky
The Tourist


model architecture: pytorch simple neural network (feedforward neural network).

get a model working before break, even if its simpler (in terms of model architecture and pre-processing).

in recommendation stage, top n choices of different artists because otherwise it will just recommend songs from the same album
|
also figure out a way to vectorize/umbrella categories for genres
|
add column for frequency of each genre like
col 1 : space rock
col 2 : rock
col 3 : frequency 3
|
also separate the words in the descriptors (ie "malevocals" should be "male vocals")

still do this, but only useful for bringing in new information that is song specific, like lyrics and popularity

In [ ]:
# CODE GRAVEYARD

# # Flatten the lists in the "descriptors" column
# flattened_descriptors = df['descriptors'].explode()

# # Count the number of unique strings
# unique_count = flattened_descriptors.nunique()

# flattened_descriptors.head(5)

# # Use value_counts to count the frequency of elements in the column
# element_counts = flattened_descriptors.value_counts()

# # Print the result
# print(element_counts)



# # One-hot-encoding for DESCRIPTORS

# # Turn descriptors into a list of strings
# df['descriptors'] = df['descriptors'].str.split(',')

# descriptors_encoded = df['descriptors'].str.join('|').str.get_dummies()

# descriptor_frequencies = descriptors_encoded.iloc[:, 2:].sum()

# # print (descriptor_frequencies)

# rarity_threshold = 10

# # Clean column names by stripping spaces
# descriptors_encoded.columns = descriptors_encoded.columns.str.strip()

# rare_descriptors = descriptor_frequencies[descriptor_frequencies < rarity_threshold]

# # print(rare_descriptors)
# # print(len(rare_descriptors))